In [ ]:
from datetime import datetime, timedelta
from typing import Callable, Any
from inspect import signature

import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
import ipywidgets as widgets
import yfinance as yf
import pandas as pd
from IPython.display import display

from openbb_terminal import api

%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")
api.theme.applyMPLstyle()

In [ ]:
model_opts = {
    "knn": api.forecast.knn,
    "mc": api.forecast.mc,
    "expo": api.forecast.expo,
    "theta": api.forecast.theta,
    "linregr": api.forecast.linregr,
    "regr": api.forecast.regr,
    "rnn": api.forecast.rnn,
    "brnn": api.forecast.brnn,
    "nbeats": api.forecast.nbeats,
    "tcn": api.forecast.tcn,
    "trans": api.forecast.trans,
    "tft": api.forecast.tft,
}

feat_engs = {
    "ema": api.forecast.ema,
    "sto": api.forecast.sto,
    "rsi": api.forecast.rsi,
    "roc": api.forecast.roc,
    "mom": api.forecast.mom,
    "atr": api.forecast.atr,
    "delta": api.forecast.delta,
    "signal": api.forecast.signal,
}

interval_opts = ["1m","2m","5m","15m","30m","60m","90m","1h","1d","5d","1wk","1mo","3mo"]

def format_df(df: pd.DataFrame) -> pd.DataFrame:
    if len(df.columns) != 6:
        df.columns = ['_'.join(col).strip() for col in df.columns.values]
    df.reset_index(inplace=True)
    df.columns = [x.lower() for x in df.columns]
    return df

def has_parameter(func: Callable[..., Any], parameter: str) -> bool:
    params = signature(func).parameters
    parameters = params.keys()
    return parameter in parameters

In [ ]:
class Chart:
    def __init__(self):
        self.last_tickers = ""
        self.last_interval = "1d"
        self.df = pd.DataFrame()
        self.infos = {}

    def handle_changes(
        self, past_covariates, start, end, interval, tickers, target_column, model, naive, forecast_only
    ):
        if tickers and tickers[-1] == ",":
            if tickers != self.last_tickers or interval != self.last_interval:
                if interval in ["1d", "5d", "1wk", "1mo", "3mo"]:
                    self.df = yf.download(
                        tickers, period="max", interval=interval, progress=False
                    )
                else:
                    end_date = end + timedelta(days=1)
                    self.df = yf.download(
                        tickers,
                        start=start,
                        end=end_date,
                        interval=interval,
                        progress=False,
                    )
                self.df = format_df(self.df)
                self.last_tickers = tickers
                self.last_interval = interval
            forecast_model = model_opts[model]
            contains_covariates = has_parameter(forecast_model, "past_covariates")
            if list(target_widget.options) != [x for x in self.df.columns if x != "date"]:
                target_widget.options = [x for x in self.df.columns if x != "date"]
                return
            if list(past_covs_widget.options) != [x for x in self.df.columns if x != "date"]:
                past_covs_widget.options = [x for x in self.df.columns if x != "date"]
                past_covs_widget.disabled = not contains_covariates
                return
            if past_covs_widget.disabled == contains_covariates:
                past_covs_widget.disabled = not contains_covariates

            start_n = datetime(start.year, start.month, start.day)
            end_n = datetime(end.year, end.month, end.day)
            calcs = self.df
            if interval in ["1d", "5d", "1wk", "1mo", "3mo"]:
                result = calcs.loc[
                    (calcs["date"] >= start_n) & (calcs["date"] <= end_n)
                ]
            else:
                result = calcs
            if not target_column:
                target_column = self.df.columns[0]
            kwargs = {}
            if contains_covariates and past_covariates != ():
                kwargs["past_covariates"] = ",".join(past_covariates)
            if has_parameter(forecast_model, "naive"):
                kwargs["naive"] = naive
            if has_parameter(forecast_model, "forecast_only"):
                kwargs["forecast_only"] = forecast_only
            # This sets up everything to run the function on button click
            self.forecast_model = forecast_model
            self.result = result
            self.target_column = target_column
            self.n_predict = 5
            self.kwargs = kwargs
            
    def handle_click(self, to_run):
        if to_run:
            run_widget.value = False
        else:
            self.forecast_model(self.result, target_column=self.target_column, n_predict=5, **self.kwargs)
            
    def handle_eng(self):
        pass

In [ ]:
w_auto = widgets.Layout(width="auto")
model_widget = widgets.Select(options=list(model_opts), value=list(model_opts)[0], layout=widgets.Layout(width="8%",))
past_covs_widget = widgets.SelectMultiple(options=[""], value=[""], layout=widgets.Layout(width="8%",))

base_date = (datetime.today() - timedelta(days=365)).date()
start_widget = widgets.DatePicker(value=base_date, layout=w_auto, description="Start")
end_widget = widgets.DatePicker(
    value=datetime.today().date(), layout=w_auto, description="End"
)

target_widget = widgets.Dropdown(options=[""], value="", description="Target")
interval_widget = widgets.Dropdown(
    options=interval_opts, value="1d", layout=w_auto, description="Interval"
)
tickers_widget = widgets.Textarea(
    value="TSLA,", layout=widgets.Layout(width="auto", height="100%")
)

naive_widget = widgets.ToggleButton(value=False, description="Show Naive")
forecast_only_widget = widgets.ToggleButton(value=False, description="Forecast Only")

run_widget = widgets.ToggleButton(value=False, description="Run Model")

selection_box = widgets.VBox([tickers_widget, target_widget])
date_box = widgets.VBox([start_widget, end_widget, interval_widget])
output_box = widgets.VBox([naive_widget, forecast_only_widget, run_widget])
controls = widgets.HBox(
    [selection_box, model_widget, past_covs_widget, date_box, output_box],
)
chart = Chart()
widgets.interactive_output(
    chart.handle_changes,
    {
        "past_covariates": past_covs_widget,
        "start": start_widget,
        "end": end_widget,
        "interval": interval_widget,
        "tickers": tickers_widget,
        "target_column": target_widget,
        "model": model_widget,
        "naive": naive_widget,
        "forecast_only": forecast_only_widget,
    },
)

stocks_view = widgets.interactive_output(chart.handle_click, {"to_run": run_widget})

title_html = "<h1>Timeseries Forecasting Dashboard</h1>"
warning_html = '<p style="color:red"=>Use a comma after EVERY stock typed.</p>'

app_contents = [widgets.HTML(title_html), controls,  widgets.HTML(warning_html), stocks_view]
app = widgets.VBox(app_contents)
display(app)